In [1]:
from google.colab import drive
drive.mount(
    '/content/drive',
    force_remount=True
)

Mounted at /content/drive


In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [3]:
import pandas as pd
import json


file_path = '/content/drive/MyDrive/senior_thesis/MT_Bench/mt_bench.json'
with open(file_path, 'r') as f:
  data = json.load(f)

ground_truth = pd.DataFrame(data)
ground_truth['id'] = ground_truth.index

ground_truth.head(5)

,input,output_1,output_2,label,id
0,Explain what's base rate fallacy and list five...,"Base rate fallacy, also known as base rate neg...",Base rate fallacy is a cognitive bias that occ...,1,0
1,"x+y = 4z, x*y = 4z^2, express x-y in z",x-y = -2z,We can start by using the formula for (x+y)^2:...,2,1
2,Please describe the concept of machine learnin...,Machine learning is a subfield of artificial i...,Machine learning is a type of artificial intel...,1,2
3,Suggest five award-winning documentary films w...,Here are five award-winning documentary films ...,1. The Cove (2009): This Academy Award-winning...,1,3
4,Benjamin went to a bookstore and purchased a v...,The cost of 5 copies of the sci-fi novel is 5 ...,* 5 sci-fi novels at $20 each = 5 * $20 = $100...,2,4


In [4]:
ground_truth.to_csv('/content/drive/MyDrive/senior_thesis/MT_Bench/mtbench_ground_truth.csv', index=True)

In [5]:
import pandas as pd
import os

folder_path = '/content/drive/MyDrive/senior_thesis/MT_Bench/outputs'
dfs = {}
for filename in os.listdir(folder_path):
  if filename.endswith('.csv'):
    print(f"Processing: {filename}")

    file_path = os.path.join(folder_path, filename)
    df = pd.read_csv(file_path)
    filename = filename[:-4]
    df['id'] = df['id'].astype(int)-1

    # df = df.set_index(['id'])
    #making zero indexing
    df['pred'] = df['pred'].astype(str).str.strip()
    #A(1) is better = 1
    df['pred'] = df['pred'].map({'A': 1, 'B': 2})
    dfs[filename] = df


Processing: gemma-1.1-2b-it.csv
Processing: gemma-2-2b-it.csv
Processing: llama-3-8b-it.csv
Processing: llama-3.2-1b-it.csv
Processing: llama-3.2-3b-it.csv
Processing: mistral-7b-it-v0.1.csv
Processing: mistral-7b-it-v0.3.csv
Processing: mistral-8b-it-2410.csv
Processing: olmo-7b-it.csv
Processing: olmoe-1b-7b-0924-it.csv
Processing: olmo-2-0325-32b-it.csv
Processing: phi-3-medium-4k-it.csv
Processing: phi-3.5-mini-it.csv
Processing: phi-4-mini-it.csv
Processing: qwen-2.5-0.5b-it.csv
Processing: qwen-2.5-1.5b-it.csv
Processing: qwen-2.5-3b-it.csv
Processing: qwen-2.5-7b-it-1m.csv
Processing: qwen-2.5-7b-it.csv
Processing: flan-t5-base.csv
Processing: flan-t5-large.csv
Processing: flan-t5-small.csv
Processing: flan-t5-xl.csv


In [6]:
dfs['flan-t5-base'].head(5)

,id,pred
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [7]:
for name, df_old in dfs.items():
    df_old = df_old.merge(ground_truth, on = 'id')  # Join on index
    df_old['correct'] = (df_old['pred'] == df_old['label']).astype(int) # Compare predictions
    dfs[name] = df_old.drop(columns=['pred', 'label'])  # Clean up by removing extra column
    # dfs[name] = df_old

In [8]:
dfs['flan-t5-xl'].head(10)

,id,input,output_1,output_2,correct
0,0,Explain what's base rate fallacy and list five...,"Base rate fallacy, also known as base rate neg...",Base rate fallacy is a cognitive bias that occ...,1
1,1,"x+y = 4z, x*y = 4z^2, express x-y in z",x-y = -2z,We can start by using the formula for (x+y)^2:...,1
2,2,Please describe the concept of machine learnin...,Machine learning is a subfield of artificial i...,Machine learning is a type of artificial intel...,1
3,3,Suggest five award-winning documentary films w...,Here are five award-winning documentary films ...,1. The Cove (2009): This Academy Award-winning...,1
4,4,Benjamin went to a bookstore and purchased a v...,The cost of 5 copies of the sci-fi novel is 5 ...,* 5 sci-fi novels at $20 each = 5 * $20 = $100...,1
5,5,"Describe a vivid and unique character, using s...","He was a giant of a man, with a gruff exterior...",The old man sat in the corner of the coffee sh...,0
6,6,You can see a beautiful red house to your left...,The White House is in the center of the three ...,The White House is located at 1600 Pennsylvani...,1
7,7,Write a function to find the majority element ...,Here's a Python function that finds the majori...,Here's a Python implementation of the Boyer-Mo...,0
8,8,Write a function to find the highest common an...,Here is a function to find the highest common ...,The highest common ancestor of two nodes in a ...,1
9,9,How many integers are in the solution of the i...,The solution of the inequality |x + 5| < 10 is...,"To solve the inequality |x + 5| < 10, we need ...",1


In [9]:
for name, df in dfs.items():
    df.to_csv(f'/content/drive/MyDrive/senior_thesis/MT_Bench/final_outputs/{name}.csv', index=False)

In [10]:
dfs.items()

dict_items([('gemma-1.1-2b-it',       id                                              input  \
0      0  Explain what's base rate fallacy and list five...   
1      1             x+y = 4z, x*y = 4z^2, express x-y in z   
2      2  Please describe the concept of machine learnin...   
3      3  Suggest five award-winning documentary films w...   
4      4  Benjamin went to a bookstore and purchased a v...   
..   ...                                                ...   
185  194  Act as a math teacher. I will provide some mat...   
186  195  Now you are a machine learning engineer. Your ...   
187  196  Embrace the role of Sheldon from "The Big Bang...   
188  197  Here is a Python function to find the length o...   
189  198  Embody the persona of Tony Stark from “Iron Ma...   

                                              output_1  \
0    Base rate fallacy, also known as base rate neg...   
1                                            x-y = -2z   
2    Machine learning is a subfield o